In [ ]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

from pympler import asizeof #Para liberar memoria
import gc

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, to_rgb


from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist

In [ ]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os

from pympler import asizeof #Para liberar memoria
import gc

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, to_rgb


from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import cdist

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_clean_afi_sel_g_all_2.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo )   

In [ ]:
data_l.shape #(62130167, 16)

In [ ]:
inicio = time.time()
data_l = data_l.sort_values( by=["CEDULA_COD","ANIO", "MES"], ascending=[ True, True, True] )
data_l.reset_index(inplace=True)
data_l.rename(columns={'index': 'nuevo_indice'}, inplace=True)
data_l.drop(columns=['nuevo_indice'], inplace=True)
data_l['INDICE'] = data_l.index

#Casos de no análisis
data_no_grupo = data_l[ (data_l['GRUPO_SEL']==0) ].copy()
data_no_grupo['ATI_CJ_M1'] = np.nan
data_no_grupo['ATI_CJ_M2'] = np.nan
data_no_grupo['ATI_CJ_M3'] = np.nan

#Casos de análisis
data = data_l[ (data_l['GRUPO_SEL']==1) ].copy()

fin = time.time()  
print('Tiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )

In [ ]:
#Para ejecutar el cluster jerarquico
def cluster_jerarquico( data_si_dic, k):
    inicio = time.time()
    nom = 'ATI_CJ_M' + str( k+1 )
    data_val_ati = {}
    for cedula in data_si_dic:

        if( len( data_si_dic[ cedula ]['SALARIO'] ) > 1 ): #Para formar al menos un cluster

            aux = np.array( data_si_dic[ cedula ]['SALARIO'] ).reshape(-1, 1)
            Z = linkage( aux , method='single', metric='euclidean')

            num_clusters = 2  # Puedes ajustar este valor según tus necesidades
            clusters = fcluster(Z, num_clusters, criterion='maxclust')

            if( len( np.unique(clusters) ) > 1 ): #Para considerar al menos 2 cluster
                # Calcular el centroide de cada clúster
                cluster_centers = np.array([[ np.nanmean(aux[clusters == i], axis=0)[0], i ] for i in range(1, num_clusters + 1)] ) 

                Q1 = np.quantile(aux, 0.25)
                Q3 = np.quantile(aux, 0.75)
                IQR = Q3-Q1
                LI = Q1 - 1.5 * IQR
                LS = Q3 + 1.5 * IQR

                cl_at = np.where( cluster_centers[:,0] > (LS +  1e-8) )[0]
                cluster_centers[cl_at][:, 1]

                mod_aux = np.zeros((len(aux), 2))
                mod_aux[:, 0] = aux[:, 0]  # Copiar los valores originales de aux en la primera columna
                mod_aux[np.isin(clusters, cluster_centers[cl_at][:, 1]), 1] = 1  # Asignar 1 en la segunda columna donde el cluster es 1

                data_val_ati[cedula] = {'SALARIO': mod_aux[:, 0].tolist(),
                                             nom : mod_aux[:, 1].tolist(),
                                        'INDICE' : data_si_dic[ cedula ]['INDICE']}
            else:
                data_val_ati[cedula] = { 'SALARIO': aux.flatten().tolist(),
                                               nom:  [-1] * len(aux),
                                         'INDICE' : data_si_dic[ cedula ]['INDICE']} 

        else:
            data_val_ati[cedula] = { 'SALARIO': data_si_dic[ cedula ]['SALARIO'],
                                           nom:  [-2] * len( data_si_dic[ cedula ]['SALARIO'] ),
                                     'INDICE' : data_si_dic[ cedula ]['INDICE']}
    fin = time.time()  
    print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    
    return data_val_ati
 
#Para unir los resultados
def extend_data( data_val_ati, k ):
    inicio = time.time()
    nom = 'ATI_CJ_M' + str( k + 1 )
    data1 = { 'CEDULA_COD': [], 'SALARIO': [], nom : [], 'INDICE':[]}

    # Llenar las listas con los datos del diccionario
    for cedula, values in data_val_ati.items():
        salario = values['SALARIO']
        atipico = values[ nom ]
        indice = values['INDICE']
        num_rows = len(salario)

        # Extender las listas en el diccionario de datos
        data1['CEDULA_COD'].extend([cedula] * num_rows)
        data1['SALARIO'].extend(salario)
        data1[ nom ].extend(atipico)
        data1['INDICE'].extend(indice)

    fin = time.time()  
    print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
    return data1

#Para graficar el comportamiento de los atipicos
def graf_ati(ced, k):
    nom = 'ATI_CJ_M'+ str(k)
    dt = data_l[(data_l['GRUPO_SEL']==1) & (data_l['CEDULA_COD']==ced)]
    unique_ati = dt[ nom ].nunique(dropna=False)
    categories = dt[ nom ].unique()

    # Generar una lista de colores discretos
    colors = plt.cm.coolwarm(np.linspace(0, 1, unique_ati))

    # Crear un mapa de colores discreto
    cmap = ListedColormap(colors)

    # Graficar
    plt.figure(figsize=(8, 5))
    scatter = plt.scatter(dt.index, dt['SALARIO'], c=dt[ nom ], cmap=cmap)

    plt.xlabel('Index')
    plt.ylabel('Salario')
    plt.title('Gráfico de Salario')

    # Crear una leyenda discreta personalizada
    # Definir las etiquetas para cada categoría
    labels = { 0: 'No es atípico', 1: 'Es atípico', -1: 'Es un solo clúter', -2:'No forma parte del proceso',
               np.nan: 'Valor NA'}

    # Crear los handles para la leyenda
    handles = [plt.Line2D([0], [0], marker='o', color='w', 
                          markersize=10, markerfacecolor=cmap(i), 
                          label=labels[i]) for i in categories]

    plt.legend(handles=handles, title='Tipo de Salario')
    plt.show()

In [ ]:
data[ ~(( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ]

In [ ]:
#Se crea un bucle para cada caso
for k in range(0,3):
    nom = 'ATI_CJ_M' + str( k+1 )
    if (k==0):
        ##Primer Caso--Considerando solo los salarios mayores al SBU y desde el año 2000
        print('*' * 50, 'Caso ', str( k+1 ), '*' * 50)
        data_si = data[   (( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() 
        data_no = data[ ~(( data['SALARIO'] >= data['SBU'] ) & ( data['ANIO'] >= 2000 )) ].copy() 
        data_no[ nom ] = np.nan
        print('Tamaño de la data utilizada dentro de los años seleccionados', data_si.shape) #(26006294, 16)
        print('Tamaño de la data no utilizada dentro de los años seleccionados',data_no.shape)  #(465822, 16)
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        
        print('Algoritmo Cluster Jerárquico')
        data_val_ati = cluster_jerarquico( data_si_dic, k )
        print('Extensión del diccionario')
        data1 = extend_data( data_val_ati, k )
        
        print('Concatenación de dataframe')
        inicio = time.time()
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO',  nom , 'INDICE']],
                                  pd.DataFrame( data1 ), 
                                  data_no[['CEDULA_COD', 'SALARIO',  nom , 'INDICE']] ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar.reset_index(inplace=True)
        data_jerar.rename(columns={'index': 'nuevo_indice'}, inplace=True)
        data_jerar.drop(columns=['nuevo_indice'], inplace=True)
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Creación de la columna')
        inicio = time.time()
        data_l[ nom ] = np.nan
        aux = data_jerar[ nom ].to_numpy()
        data_l.iloc[:, data_l.columns.get_loc( nom )] = aux
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Verificación de posibles errores')
        inicio = time.time()
        dif1 = data_l[data_l['SALARIO']!= data_jerar['SALARIO']].index
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        print('*' * 108, '\n')  
        
    if (k==1):
        ##Segundo Caso--Considerando todo el registro de los mejores años
        print('*' * 50, 'Caso ', str( k+1 ), '*' * 50)
        data_si = data.copy()
        print('Tamaño de la data utilizada dentro de los años seleccionados', data_si.shape) 
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        
        print('Algoritmo Cluster Jerárquico')
        data_val_ati = cluster_jerarquico( data_si_dic, k )
        print('Extensión del diccionario')
        data1 = extend_data( data_val_ati, k )
        
        print('Concatenación de dataframe')
        inicio = time.time()
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', nom, 'INDICE']],
                                  pd.DataFrame( data1 ) ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar.reset_index(inplace=True)
        data_jerar.rename(columns={'index': 'nuevo_indice'}, inplace=True)
        data_jerar.drop(columns=['nuevo_indice'], inplace=True)
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Creación de la columna')
        inicio = time.time()
        data_l[ nom ] = np.nan
        aux = data_jerar[ nom ].to_numpy()
        data_l.iloc[:, data_l.columns.get_loc( nom ) ] = aux
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Verificación de posibles errores')
        inicio = time.time()
        dif2 = data_l[data_l['SALARIO']!= data_jerar['SALARIO']].index
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        print('*' * 108, '\n')
        
    if (k==2):
        ##Tercer Caso--Considerando todo el registro de los mejores años, pero a partir del año 2000
        print('*' * 50, 'Caso ', str( k+1 ), '*' * 50)
        data_si = data[  ( ( data['ANIO'] >= 2000 )) ].copy()
        data_no = data[ ~( ( data['ANIO'] >= 2000 )) ].copy()   
        data_no[ nom ] = np.nan
        print('Tamaño de la data utilizada dentro de los años seleccionados', data_si.shape) 
        print('Tamaño de la data no utilizada dentro de los años seleccionados',data_no.shape) 
        
        data_si_dic = data_si.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        data_no_dic = data_no.groupby('CEDULA_COD').agg({'SALARIO': list, 'INDICE': list}).to_dict(orient='index')
        
        print('Algoritmo Cluster Jerárquico')
        data_val_ati = cluster_jerarquico( data_si_dic, k )
        print('Extensión del diccionario')
        data1 = extend_data( data_val_ati, k )
        
        print('Concatenación de dataframe')
        inicio = time.time()
        data_jerar = None
        data_jerar = pd.concat( [ data_no_grupo[['CEDULA_COD', 'SALARIO', nom, 'INDICE']],
                                  pd.DataFrame( data1 ), 
                                  data_no[['CEDULA_COD', 'SALARIO', nom, 'INDICE']] ], axis=0)
    
        data_jerar = data_jerar.sort_values( by=["INDICE"], ascending=[ True ] )
        data_jerar.reset_index(inplace=True)
        data_jerar.rename(columns={'index': 'nuevo_indice'}, inplace=True)
        data_jerar.drop(columns=['nuevo_indice'], inplace=True)
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Creación de la columna')
        inicio = time.time()
        data_l[ nom ] = np.nan
        aux = data_jerar[ nom ].to_numpy()
        data_l.iloc[:, data_l.columns.get_loc( nom )] = aux
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        
        print('Verificación de posibles errores')
        inicio = time.time()
        dif3 = data_l[data_l['SALARIO']!= data_jerar['SALARIO']].index
        fin = time.time()  
        print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos' )
        print('*' * 108, '\n')


In [ ]:
del data, data_jerar, data1, aux, data_si, data_no, data_no_grupo, data_val_ati
gc.collect()

In [ ]:
#Se calculan los limítes máximos de aportación en función de la HL
#Caso 1: Para toda la historia laboral
inicio = time.time()   
grupo1 = data_l.copy()
Q1 = grupo1.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
Q3 = grupo1.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR
data_l['LS1'] = data_l['CEDULA_COD'].map( LS )

#Caso 2: Para la historia laboral a partir del año 2020
grupo2 = data_l[ data_l['ANIO']>=2000 ].copy()
Q1 = grupo2.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
Q3 = grupo2.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR
data_l['LS2'] = data_l['CEDULA_COD'].map( LS )

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos')

del grupo1, grupo2, Q1, Q3, IQR, LI, LS
gc.collect()

In [ ]:
#El valor de NA implica que esos datos no son parte de data_si, por lo que no se analizaron
#El valor de -1 implica que se formó un solo cluster
#El valor de -2 implica que en la selección de datos (data_si), la cedula solo tenía un único valor para formar los cluster
# pues era el resultado de filtrar el data_si

##Se analiza el caso de los valores de atipicos del método 1
# data_l[ ( data_l['GRUPO_SEL']==1 )]['ATI_CJ_M1'].unique() #array([ 0., nan,  1., -1., -2.])
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M1']==-2)] #86 rows × 21 columns
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M1'].isna())]  #465822 rows × 21 columns
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M1']==-1)]  # 3292785 rows × 21 columns

##Se analiza el caso de los valores de atipicos del método 2
# data_l[ ( data_l['GRUPO_SEL']==1 )]['ATI_CJ_M2'].unique() #rray([ 0.,  1., -1.])
# ata_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M2']==-1)]  #3125765 rows × 21 columns

##Se analiza el caso de los valores de atipicos del método 3
# data_l[ ( data_l['GRUPO_SEL']==1 )]['ATI_CJ_M3'].unique() #array([ 0.,  1., -1., nan, -2.])
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M3']==-2)] #2 rows × 21 columns
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M3'].isna())]  # 7454 rows × 21 columns
# data_l[ ( data_l['GRUPO_SEL']==1 ) & (data_l['ATI_CJ_M3']==-1)] #3125140 rows × 21 columns

In [ ]:
#Cedulas que tiene datos atípicos----------------------------------------------------------------------------------------------
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique()) #134017
print('\tCedulas con valor -2:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-2)]['CEDULA_COD'].nunique()) #86
print('\tCedulas con valor nan:',           data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1'].isna())]['CEDULA_COD'].nunique()) #85804
print('\tCedulas con valor -1:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-1)]['CEDULA_COD'].nunique()) #55669
print('\tCedulas con valor 0:',             data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==0)]['CEDULA_COD'].nunique()) #379216
print('*' * 50, 'Caso ', str( 2 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==1)]['CEDULA_COD'].nunique()) #123850
print('\tCedulas con valor -2:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==-2)]['CEDULA_COD'].nunique()) #0
print('\tCedulas con valor nan:',           data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2'].isna())]['CEDULA_COD'].nunique()) #0
print('\tCedulas con valor -1:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==-1)]['CEDULA_COD'].nunique()) #52121
print('\tCedulas con valor 0:',             data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==0)]['CEDULA_COD'].nunique()) #383682
print('*' * 50, 'Caso ', str( 3 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==1)]['CEDULA_COD'].nunique()) #123826
print('\tCedulas con valor -2:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-2)]['CEDULA_COD'].nunique()) #2
print('\tCedulas con valor nan:',           data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3'].isna())]['CEDULA_COD'].nunique()) #499
print('\tCedulas con valor -1:',            data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-1)]['CEDULA_COD'].nunique()) # 52124
print('\tCedulas con valor 0:',             data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==0)]['CEDULA_COD'].nunique()) #383653
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)

In [ ]:
graf_ati(23075971, 1)

In [ ]:
#Caso 1- Correcciones---------------------------------------------------------------------------------------------------
#Se corrige en función deL Límite máximo de la HL a apartir del año 2000, pues es la fecha en la que entra en dolar a ecuador ( -2 -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-2) & (data_l['SALARIO'] > data_l['LS2']) ), 'ATI_CJ_M1'] = 1 #quedan 45 casos
#Se corrige en función del valores menoRes al limite máximo la HL a apartir del año 2000, pues es la fecha en la que entra en dolar a ecuador ( -2 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-2) & (data_l['SALARIO'] <= data_l['LS2']) ), 'ATI_CJ_M1'] = 0 
# Se hacen las correcciones para los valores -2, en donde si el Salario es menor o igual a un SBU no es atípico( -2 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-2) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M1'] = 0 

#Se corrige en función deL Límite máximo de toda la HL, pues si no hay datos del SBU para atrás ( NA -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1'].isna()) & (data_l['SALARIO'] > data_l['LS1']) ), 'ATI_CJ_M1'] = 1  #quedan 463235 casos
#Se corrige en función del valores menoRes al limite máximo la HL, pues si no hay datos del SBU para atrás ( NA -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1'].isna()) & (data_l['SALARIO'] <= data_l['LS1']) ), 'ATI_CJ_M1'] = 0  #
#Se hacen las correcciones para los valores NA, en donde si el Salario es menor o igual a un SBU no es atípico( NA -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1'].isna()) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M1'] = 0  

#Se hacen las correcciones para los valores -1, en donde si el Salario es mayor a LS1 es atípico( -1 -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-1) & (data_l['SALARIO'] > data_l['LS1']) ), 'ATI_CJ_M1'] = 1 #quedan 3267088
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a LS1 no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-1) & (data_l['SALARIO'] <= data_l['LS1']) ), 'ATI_CJ_M1'] = 0 #q 
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a un SBU no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==-1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M1'] = 0 #

#Se hacen las correcciones para los valores 1, en donde si el Salario es menor o igual al SBU no es atípico( 1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M1'] = 0

In [ ]:
#Caso 2 - Correcciones---------------------------------------------------------------------------------------------------
#No existen valores igual a NA e igual a -2
#Se hacen las correcciones para los valores -1, en donde si el Salario es mayor a LS1 es atípico( -1 -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==-1) & (data_l['SALARIO'] > data_l['LS1']) ), 'ATI_CJ_M2'] = 1 # quedan 3104185
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a LS1 no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==-1) & (data_l['SALARIO'] <= data_l['LS1']) ), 'ATI_CJ_M2'] = 0
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a un SBU no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==-1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M2'] = 0 
#Se hacen las correcciones para los valores 1, en donde si el Salario es menor o igual al SBU no es atípico( 1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M2'] = 0

In [ ]:
#Caso 3 - Correcciones-------------------------------------------------------------------------------------------------
#Se corrige en función deL Límite máximo de la HL a apartir del año 2000, pues es la fecha en la que entra en dolar a ecuador ( -2 -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-2) & (data_l['SALARIO'] > data_l['LS2']) ), 'ATI_CJ_M3'] = 1 # quedan 2 casos
#Se corrige en función del valores menores al limite máximo la HL a apartir del año 2000, pues es la fecha en la que entra en dolar a ecuador ( -2 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-2) & (data_l['SALARIO'] <= data_l['LS2']) ), 'ATI_CJ_M3'] = 0
#Se hacen las correcciones para los valores -2, en donde si el Salario es menor o igual a un SBU no es atípico( -2 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-2) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M3'] = 0 #

#Se corrige en función deL Límite máximo de toda la HL, pues si no hay datos del SBU para atrás ( NA -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3'].isna()) & (data_l['SALARIO'] > data_l['LS1']) ), 'ATI_CJ_M3'] = 1 #quedan 7394 casos
#Se corrige en función del valores menoRes al limite máximo la HL, pues sno hay datos del SBU para atrás ( NA -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3'].isna()) & (data_l['SALARIO'] <= data_l['LS1']) ), 'ATI_CJ_M3'] = 0  
#Se hacen las correcciones para los valores NA, en donde si el Salario es menor o igual a un SBU no es atípico( NA -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3'].isna()) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M3'] = 0 

#Se hacen las correcciones para los valores -1, en donde si el Salario es mayor a LS1 es atípico( -1 -> 1 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-1) & (data_l['SALARIO'] > data_l['LS1']) ), 'ATI_CJ_M3'] = 1 #quedan 3103457  casos
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a LS1 no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-1) & (data_l['SALARIO'] <= data_l['LS1']) ), 'ATI_CJ_M3'] = 0
#Se hacen las correcciones para los valores -1, en donde si el Salario es menor o igual a un SBU no es atípico( -1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==-1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M3'] = 0 
#Se hacen las correcciones para los valores 1, en donde si el Salario es menor o igual al SBU no es atípico( 1 -> 0 )
data_l.loc[ ( (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==1) & (data_l['SALARIO'] <= data_l['SBU']) ), 'ATI_CJ_M3'] = 0

In [ ]:
graf_ati(23075971, 1)

In [ ]:
#Cedulas que tiene datos atípicos----------------------------------------------------------------------------------------------
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique()) #125237
print('\tCedulas con sin atípicos:',        data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==0)]['CEDULA_COD'].nunique()) #436149
print('*' * 50, 'Caso ', str( 2 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==1)]['CEDULA_COD'].nunique()) #116931
print('\tCedulas con sin atípicos:',        data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==0)]['CEDULA_COD'].nunique()) #436148
print('*' * 50, 'Caso ', str( 3 ), '*' * 50)
print('\tCedulas con al menos un atípico:', data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==1)]['CEDULA_COD'].nunique()) #116938
print('\tCedulas con sin atípicos:',        data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==0)]['CEDULA_COD'].nunique()) #436149
print('*' * 102)

In [ ]:
# Guadar archivo------------------------------------------------------------------------------------------------------------
nombre_archivo = 'viu_clean_data_l.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Objetos a guardar
objeto8 = data_l

# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(objeto8, archivo)      

del objeto8

In [ ]:
data_l.shape

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_clean_data_l.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_l = pickle.load( archivo ) 

In [ ]:
#Analisis para atípicos vs aportes simultáneos
cedula_1 = data_l.groupby('CEDULA_COD')['NUM_SEC_MES'].apply( lambda x: (x != 1).any() )
cedula_dist = cedula_1[ cedula_1 ].index
ul = data_l[ ~data_l['CEDULA_COD'].isin( cedula_dist )]
ml = data_l[  data_l['CEDULA_COD'].isin( cedula_dist )]

#Cedulas que tiene datos atípicos----------------------------------------------------------------------------------------------
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      ul[ (ul['GRUPO_SEL']==1) & (ul['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique())
print('*' * 50, 'Caso ', str( 2 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      ul[ (ul['GRUPO_SEL']==1) & (ul['ATI_CJ_M2']==1)]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M2']==1)]['CEDULA_COD'].nunique())
print('*' * 50, 'Caso ', str( 3 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      ul[ (ul['GRUPO_SEL']==1) & (ul['ATI_CJ_M3']==1)]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M3']==1)]['CEDULA_COD'].nunique())
print('*' * 102)

In [ ]:
#Creando limites de atipicos
inicio = time.time()   
#Caso 3: Para toda la historia laboral sin considerar aportes simultáneos
# cedula_1 = data_l.groupby('CEDULA_COD')['NUM_SEC_MES'].apply( lambda x: (x != 1).any() )
# cedula_dist = cedula_1[ cedula_1 ].index
# grupo3 = data_l[ ~data_l['CEDULA_COD'].isin( cedula_dist )]
grupo3 = ul

Q1 = grupo3.groupby('CEDULA_COD')['SALARIO'].quantile(0.25)
Q3 = grupo3.groupby('CEDULA_COD')['SALARIO'].quantile(0.75)
IQR = Q3-Q1
LI = Q1 - 1.5 * IQR
LS = Q3 + 1.5 * IQR
data_l['LS3'] = data_l['CEDULA_COD'].map( LS )

#Caso 4: Para toda la historia laboral considerando aportes simultáneos
#grupo4 = data_l[ data_l['CEDULA_COD'].isin( cedula_dist )]
grupo4 = ml
dic_aux = grupo4.groupby('CEDULA_COD').agg({'SALARIO_SECTOR': list, 'INDICE': list}).to_dict(orient='index')

for cedula in dic_aux:
    dic_aux[cedula]['SALARIO_SECTOR'] = [ [float(val) for val in sal.replace(':', ';').split(';')] 
                                            if isinstance(sal, str) 
                                            else [float(sal)]
                                            for sal in dic_aux[cedula]['SALARIO_SECTOR']
                                        ]
    # Aplanar la lista de listas de SALARIO_SECTOR
    salarios = None
    salarios = [salario for sublist in dic_aux[cedula]['SALARIO_SECTOR'] for salario in sublist]
    dic_aux[cedula]['Q1'] =  np.percentile(salarios, 25)
    dic_aux[cedula]['Q3'] =  np.percentile(salarios, 75)
    dic_aux[cedula]['IQR'] = dic_aux[cedula]['Q3'] -  dic_aux[cedula]['Q1']
    dic_aux[cedula]['LI'] =  dic_aux[cedula]['Q1'] -  1.5  * dic_aux[cedula]['IQR']
    dic_aux[cedula]['LS'] =  dic_aux[cedula]['Q3'] +  1.5  * dic_aux[cedula]['IQR']

data1 = { 'CEDULA_COD': [], 'LS': [] }

for cedula, val in dic_aux.items():
    data1['CEDULA_COD'].append(cedula)
    data1['LS'].append( val['LS'] )

LS = pd.DataFrame( data1 )
LS = LS.groupby('CEDULA_COD')['LS'].first()
data_l['LS4'] = data_l['CEDULA_COD'].map( LS )  

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos')

del grupo3, grupo4, dic_aux, data1, Q1, Q3, IQR, LI, LS
gc.collect()

In [ ]:
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==1) ]['CEDULA_COD'].unique()

In [ ]:
data_l[ (data_l['GRUPO_SEL']==1) & (data_l['CEDULA_COD']==397)]

In [ ]:
del ul, ml
gc.collect()

In [ ]:
graf_ati(397, 1)

In [ ]:
data_l['ATI_CJ_AS'] = 0
grupo1 = None
grupo1 = data_l[  data_l['CEDULA_COD'].isin( cedula_dist )].copy() #Es igual a ml pero se actualizan los valores

inicio = time.time()   
dic_sim = grupo1.groupby('CEDULA_COD').agg({'SALARIO_SECTOR': list, 'LS4': list,'INDICE': list}).to_dict(orient='index')

for cedula in dic_sim:
    dic_sim[cedula]['SALARIO_SECTOR'] = [[float(val) for val in sal.split(';')] if isinstance(sal, str) else [sal] 
                                          for sal in dic_sim[cedula]['SALARIO_SECTOR']]
    dic_sim[cedula]['ATI_CJ_AS'] = [ 1 if any(sal > ls for sal in salarios) else 0 
                                    for salarios, ls in zip( dic_sim[cedula]['SALARIO_SECTOR'], dic_sim[cedula]['LS4'])]

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos')

del grupo1   
gc.collect()

In [ ]:
data1 = { 'CEDULA_COD': [], 'SALARIO_SECTOR': [], 
         'LS4' : [], 'ATI_CJ_AS':[],  'INDICE':[]}

# Llenar las listas con los datos del diccionario
for cedula, values in dic_sim.items():
    salario = values['SALARIO_SECTOR']
    ls = values[ 'LS4' ]
    atipico = values[ 'ATI_CJ_AS' ]
    indice = values['INDICE']
    num_rows = len(salario)

    # Extender las listas en el diccionario de datos
    data1['CEDULA_COD'].extend([cedula] * num_rows)
    data1['SALARIO_SECTOR'].extend(salario)
    data1['LS4' ].extend(ls)
    data1['ATI_CJ_AS' ].extend(atipico)
    data1['INDICE'].extend(indice)
    
data1 = pd.DataFrame( data1 )
data1.set_index('INDICE', inplace=True )
data1

In [ ]:
inicio = time.time()   
cedulas = list(dic_sim.keys())  #total de 111903
filtro = data1[ data1['CEDULA_COD'].isin(cedulas )] # 111903 cedulas
indi = filtro.index
data_l.loc[ indi, 'ATI_CJ_AS'] = filtro['ATI_CJ_AS']

fin = time.time()  
print('\tTiempo de ejecución es: ',  (fin-inicio)//3600, ' horas con ' ,  (fin-inicio)%3600//60 , ' minutos y', (fin-inicio)%60, ' segundos')

In [ ]:
sal=[[float(val) for val in sal.split(';')] if isinstance(sal, str) else [sal] for sal in data_l[data_l['CEDULA_COD']==216]['SALARIO_SECTOR']]
sala=[salario for sublist in sal for salario in sublist]
LS = np.percentile(sala, 75) + 1.5 * (np.percentile(sala, 75)-np.percentile(sala, 25))
LS

In [ ]:
data_l[data_l['CEDULA_COD']==216].head(60)

In [ ]:
data1[data1['CEDULA_COD']==216].head(60)

In [ ]:
ml = data_l[  data_l['CEDULA_COD'].isin( cedula_dist )].copy()
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)
print('\tCedulas que tienen más de un sector y que fue clasificado como atípico, que se modificaran a no atipico:',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==1) & ( ml['ATI_CJ_AS']==0)]['CEDULA_COD'].nunique()) #29408
print('\tCedulas que tienen más de un sector y que fue clasificado como  no atípico, que se modificaran a atipico',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==0) & ( ml['ATI_CJ_AS']==1)]['CEDULA_COD'].nunique()) #24171

print('*' * 50, 'Caso ', str( 2 ), '*' * 50)
print('\tCedulas que tienen más de un sector y que fue clasificado como atípico, que se modificarann a no atipico:',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M2']==1) & ( ml['ATI_CJ_AS']==0)]['CEDULA_COD'].nunique()) #26929
print('\tCedulas que tienen más de un sector y que fue clasificado como  no atípico, que se modificaran a atipico',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M2']==0) & ( ml['ATI_CJ_AS']==1)]['CEDULA_COD'].nunique()) #24893

print('*' * 50, 'Caso ', str( 3 ), '*' * 50)
print('\tCedulas que tienen más de un sector y que fue clasificado como atípico, que se modificarann a no atipico:',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M3']==1) & ( ml['ATI_CJ_AS']==0)]['CEDULA_COD'].nunique()) #26933
print('\tCedulas que tienen más de un sector y que fue clasificado como  no atípico, que se modificaran a atipico',
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M3']==0) & ( ml['ATI_CJ_AS']==1)]['CEDULA_COD'].nunique()) #24893
print('*' * 102)

In [ ]:
#Cedulas que tiene varios sectores que se modificaran ( 1 -> 0 )
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==1) & ( ml['ATI_CJ_AS']==0)]['CEDULA_COD'].unique() #29408

In [ ]:
#Cedulas que tiene varios sectores que se modificaran ( 0 -> 1 )
ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==0) & ( ml['ATI_CJ_AS']==1)]['CEDULA_COD'].unique() #24171

In [ ]:
ml[ (ml['GRUPO_SEL']==1) & (ml['CEDULA_COD']==1908)]

In [ ]:
#Gráficas para cuando de 1->0 en aportes simultáneos
graf_ati(1908, 1)

In [ ]:
#Gráficas para cuando de 0->1 en aportes simultáneos
ml[ (ml['GRUPO_SEL']==1) & (ml['CEDULA_COD']==472)]

In [ ]:
#Gráficas para cuando de 0 -> 1 en aportes simultáneos
graf_ati(472, 1)

In [ ]:
cedula_dist 

In [ ]:
#Se hacen las correcciones
#Caso 1
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M1'] == 1) & (data_l['ATI_CJ_AS'] == 0), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M1'] == 0) & (data_l['ATI_CJ_AS'] == 1), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']

#Caso 2
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M2'] == 1) & (data_l['ATI_CJ_AS'] == 0), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M2'] == 0) & (data_l['ATI_CJ_AS'] == 1), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']

#Caso 3
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M3'] == 1) & (data_l['ATI_CJ_AS'] == 0), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']
data_l.loc[(data_l['CEDULA_COD'].isin(cedula_dist)) & 
           (data_l['GRUPO_SEL'] == 1) & (data_l['ATI_CJ_M3'] == 0) & (data_l['ATI_CJ_AS'] == 1), 'ATI_CJ_M1'] = data_l['ATI_CJ_AS']


In [ ]:
del data1, dic_sim, filtro, indi, cedulas, ml
gc.collect()

In [ ]:
#Cedulas que tiene datos atípicos luego de la corrección---------------------------------------------------------------------
print('*' * 50, 'Caso ', str( 1 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==1) &(~data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M1']==1) & (data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique())
print('*' * 50, 'Caso ', str( 2 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==1) &(~data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M2']==1) &(data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique())
print('*' * 50, 'Caso ', str( 3 ), '*' * 50)
print('\tCedulas con al menos un atípico con un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==1) &(~data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique()) 
print('\tCedulas con al menos un atípico con más de un único sector:',
      data_l[ (data_l['GRUPO_SEL']==1) & (data_l['ATI_CJ_M3']==1) &(data_l['CEDULA_COD'].isin(cedula_dist))]['CEDULA_COD'].nunique())
print('*' * 102)

In [ ]:
ml = data_l[  data_l['CEDULA_COD'].isin( cedula_dist )]
ul = data_l[ ~ data_l['CEDULA_COD'].isin( cedula_dist )]

In [ ]:
print(ul[ (ul['GRUPO_SEL']==1) & (ul['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique())
print(ml[ (ml['GRUPO_SEL']==1) & (ml['ATI_CJ_M1']==1)]['CEDULA_COD'].nunique())

In [ ]:
graf_ati(1908, 1)

In [ ]:
graf_ati(472, 1)